In [85]:
import pandas as pd
import pickle

In [86]:
geneformer_gene_pairs = pd.read_pickle('gf_split_outputs/gene_pairs.pkl').reset_index()
geneformer_gene_pairs['GeneAB'] = geneformer_gene_pairs['Gene_A'] + '_' + geneformer_gene_pairs['Gene_B']

In [87]:
geneformer_gene_pairs

,index,Gene_A,Gene_B,GeneAB
0,0,ST13P4,WASH2P,ST13P4_WASH2P
1,1,ST13P4,FAM90A12P,ST13P4_FAM90A12P
2,2,ST13P4,PI4KAP1,ST13P4_PI4KAP1
3,3,ST13P4,SNX29P2,ST13P4_SNX29P2
4,4,ST13P4,ZNF818P,ST13P4_ZNF818P
...,...,...,...,...
200794870,200794870,RBMXP1,PPT1,RBMXP1_PPT1
200794871,200794871,RBMXP1,PLD2,RBMXP1_PLD2
200794872,200794872,RBMXP1,EIF4EBP1,RBMXP1_EIF4EBP1
200794873,200794873,RBMXP1,TINAG,RBMXP1_TINAG


In [6]:
GF_6l30m = pd.read_pickle('gf_split_outputs/GF-6L30M_HUMANemb_similarities_similarity_only.pkl').reset_index().dropna()

In [7]:
gf_pairs = geneformer_gene_pairs.merge(GF_6l30m, on = 'index')[['index', 'Gene_A', 'Gene_B']]
gf_pairs['GeneAB'] = gf_pairs['Gene_A'] + '_' + gf_pairs['Gene_B']

In [8]:
del GF_6l30

In [90]:
complex_labels = pd.read_pickle('complex_label.pkl')

In [91]:
complex_labels

,GeneAB,Same_Complex
0,ST13P4_WASH2P,0
2,ST13P4_FAM90A12P,0
3,ST13P4_PI4KAP1,0
4,ST13P4_SNX29P2,0
5,ST13P4_ZNF818P,0
...,...,...
325245048,RBMXP1_PPT1,0
325245049,RBMXP1_PLD2,0
325245050,RBMXP1_EIF4EBP1,0
325245051,RBMXP1_TINAG,0


In [88]:
geneformer_gene_pairs.iloc[174701]

index          174701
Gene_A          NBPF8
Gene_B          PYDC1
GeneAB    NBPF8_PYDC1
Name: 174701, dtype: object

In [92]:
complex_labels.iloc[174701]

GeneAB          NBPF8_PYDC1
Same_Complex              0
Name: 223203, dtype: object

In [93]:
scgpt_human = pd.read_pickle('scgpt_split_outputs/scGPT_human_embeddings_similarities_similarity_only.pkl')
scgpt_gp = pd.read_pickle('scgpt_split_outputs/gene_pairs.pkl')
scgpt_human = pd.concat([scgpt_gp, scgpt_human], axis=1)
scgpt_human

,Gene_A,Gene_B,Cosine_Similarity
0,ST13P4,WASH2P,0.357125
1,ST13P4,FAM90A12P,0.106240
2,ST13P4,PI4KAP1,0.160441
3,ST13P4,SNX29P2,0.211458
4,ST13P4,ZNF818P,0.100117
...,...,...,...
200794870,RBMXP1,PPT1,-0.122569
200794871,RBMXP1,PLD2,0.069859
200794872,RBMXP1,EIF4EBP1,-0.037521
200794873,RBMXP1,TINAG,0.135589


In [94]:
scgpt_human.iloc[174701]

Gene_A                  NBPF8
Gene_B                  PYDC1
Cosine_Similarity    0.183128
Name: 174701, dtype: object

In [11]:
all_labels = gf_pairs.merge(complex_labels, on = 'GeneAB')[['index', 'GeneAB', 'Same_Complex']]

In [12]:
all_labels

,index,GeneAB,Same_Complex
0,174701,NBPF8_PYDC1,0
1,174702,NBPF8_PYCR3,0
2,174703,NBPF8_BAG3,0
3,174704,NBPF8_PXT1,0
4,174705,NBPF8_PXMP4,0
...,...,...,...
175489836,200777371,BCAT1_GPR55,0
175489837,200777372,BCAT1_CEMIP,0
175489838,200777373,BCAT1_IL4R,0
175489839,200777375,BCAT1_THSD7A,0


In [13]:
all_labels.to_pickle('all_labels_og_index.pkl')

In [1]:
##### restarting kernel

In [71]:
from sklearn.model_selection import StratifiedKFold
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_curve, precision_recall_curve, auc
from pathlib import Path

def stratified_cv_return_indices(index_df, label_col='Label', index_col='index', n_splits=5, random_state=42, mode='balanced'):
    """
    Performs stratified CV and returns lists of train/test index values for each fold.
    
    Parameters:
        index_df (pd.DataFrame): Must contain at least 'index' and label column.
        label_col (str): Name of the label column.
        index_col (str): Name of the column with numeric index values used for subsetting.
        n_splits (int): Number of cross-validation folds.
        random_state (int): Random seed for reproducibility.
        mode (str): One of 'balanced' or 'full_neg_train'

    Returns:
        List of (train_idx_list, test_idx_list) tuples (lists of numeric indices).
    """
    pos_df = index_df[index_df[label_col] == 1].copy()
    neg_df = index_df[index_df[label_col] == 0].copy()

    skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=random_state)
    folds = list(skf.split(pos_df, pos_df[label_col]))

    fold_indices = []

    for fold_idx, (train_pos_idx, test_pos_idx) in enumerate(folds):
        train_pos = pos_df.iloc[train_pos_idx]
        test_pos = pos_df.iloc[test_pos_idx]

        if mode == 'balanced':
            train_neg = neg_df.sample(n=len(train_pos), random_state=random_state + fold_idx)
            test_neg = neg_df.drop(train_neg.index, errors='ignore').sample(
                n=len(test_pos), random_state=random_state + 10 + fold_idx
            )
            train_df = pd.concat([train_pos, train_neg])
            train_idx = pd.concat([train_pos, train_neg])[index_col].tolist()
            test_df = pd.concat([test_pos, test_neg]) 
            test_idx = pd.concat([test_pos, test_neg])[index_col].tolist()

            train_df.to_csv(f'balanced_train_{fold_idx}_labels.csv')
            test_df.to_csv(f'balanced_test_{fold_idx}_labels.csv')
            

        elif mode == 'full_neg_train':
            train_idx = pd.concat([train_pos, neg_df])[index_col].tolist()
            test_neg = neg_df.sample(n=len(test_pos), random_state=random_state + 10 + fold_idx)
            test_idx = pd.concat([test_pos, test_neg])[index_col].tolist()

        else:
            raise ValueError("mode must be 'balanced' or 'full_neg_train'")

        fold_indices.append((train_idx, test_idx))

    return fold_indices

def load_features_by_index(feature_files, index_subset):
    """
    Loads data from a list of feature files and returns a merged dataframe,
    renaming 'Cosine_Similarity' columns to avoid collisions,
    and adding correlation column from a special spearman file.

    Parameters:
        feature_files (List[str or Path]): List of .pkl file paths.
        index_subset (List[int]): List of numeric indices to keep.

    Returns:
        pd.DataFrame: Merged dataframe with one column per file (renamed as needed).
    """
    collected = []
    index_subset_set = set(index_subset)

    for file in feature_files:
        file_path = Path(file)
        f = file.split('/')[-1]
        df = pd.read_pickle(file_path)

        # Subset by index
        df_subset = df[df.index.isin(index_subset_set)].copy()

        if file_path.name == "allpairs_spearman_correlation.pkl":
            # Use the 'correlation' column as-is (assumes it's the second column)
            if 'Correlation' not in df_subset.columns:
                raise ValueError(f"'correlation' column not found in {file_path.name}")
            collected.append(df_subset[['Correlation']])
        else:
            # Handle cosine similarity renaming
            prefix = f.replace('_similarities_similarity_only.pkl', '')
            new_col_name = f"{prefix}_Cosine_Similarity"
            df_subset.rename(columns={"Cosine_Similarity": new_col_name}, inplace=True)
            collected.append(df_subset[[new_col_name]])

    # Merge all columns on index
    merged_df = pd.concat(collected, axis=1)

    return merged_df

In [2]:
import pandas as pd
all_labels = pd.read_pickle('all_labels_og_index.pkl')

In [4]:
import os

In [54]:
scgpt = ['scgpt_split_outputs/'+f for f in os.listdir('scgpt_split_outputs') if f.endswith('similarity_only.pkl')]
gf = ['gf_split_outputs/'+f for f in os.listdir('gf_split_outputs') if f.endswith('similarity_only.pkl')]

In [55]:
scgpt

['scgpt_split_outputs/scGPT_bc_embeddings_similarities_similarity_only.pkl',
 'scgpt_split_outputs/scGPT_pancancer_embeddings_similarities_similarity_only.pkl',
 'scgpt_split_outputs/scGPT_CP_embeddings_similarities_similarity_only.pkl',
 'scgpt_split_outputs/scGPT_lung_embeddings_similarities_similarity_only.pkl',
 'scgpt_split_outputs/scGPT_heart_embeddings_similarities_similarity_only.pkl',
 'scgpt_split_outputs/scGPT_brain_embeddings_similarities_similarity_only.pkl',
 'scgpt_split_outputs/scGPT_kidney_embeddings_similarities_similarity_only.pkl',
 'scgpt_split_outputs/scGPT_human_embeddings_similarities_similarity_only.pkl']

In [56]:
gf

['gf_split_outputs/GF-6L30M_HUMANemb_similarities_similarity_only.pkl',
 'gf_split_outputs/GF-20L95M_HUMANemb_similarities_similarity_only.pkl',
 'gf_split_outputs/GF-12L95M_HUMANemb_similarities_similarity_only.pkl',
 'gf_split_outputs/GF-12L95MCANCER_UNIPROT_HUMANemb_similarities_similarity_only.pkl',
 'gf_split_outputs/GF-12L30M_HUMANemb_similarities_similarity_only.pkl']

In [13]:
all_labels.columns

Index(['index', 'GeneAB', 'Same_Complex'], dtype='object')

In [72]:
# Step 2: Get fold index lists
fold_indices = stratified_cv_return_indices(all_labels, label_col = 'Same_Complex', index_col = 'index')

In [95]:
fold_indices

[([1233514,
   1236971,
   1238154,
   1248692,
   1704555,
   1704560,
   1705712,
   1706673,
   1706709,
   1707847,
   1709869,
   1710406,
   1711159,
   1711438,
   1712455,
   1713163,
   1713550,
   1715692,
   1715758,
   1716246,
   1719473,
   1721062,
   1722762,
   1722856,
   1723022,
   1723403,
   1724539,
   1725176,
   1726245,
   1728907,
   1729361,
   1731354,
   1731921,
   1733107,
   1733258,
   1754698,
   1768427,
   1787933,
   1789409,
   1791198,
   1791547,
   1793498,
   1793707,
   1794503,
   1794781,
   1794785,
   1795732,
   1795857,
   1796753,
   1796788,
   1800253,
   1800958,
   1801217,
   1802158,
   1802805,
   1804095,
   1804437,
   1805223,
   1805680,
   1807883,
   1808227,
   1809690,
   1810154,
   1811811,
   1811966,
   2212971,
   2214301,
   2219098,
   2219494,
   2222309,
   2224357,
   2226321,
   2226891,
   2228612,
   2230419,
   2469369,
   2532582,
   2805770,
   2807898,
   2820847,
   2952775,
   3068316,
   3069722,
   3

In [35]:
import pickle

# Save to file
with open("balanced_5fold_indices.pkl", "wb") as f:
    pickle.dump(fold_indices, f)

In [18]:
mini_gf = ['GF-6L30M_HUMANemb_similarities_similarity_only.pkl',
 'GF-20L95M_HUMANemb_similarities_similarity_only.pkl']

minigf = ['./gf_split_outputs/' + f for f in mini_gf]

In [42]:
minigf.append('allpairs_spearman_correlation.pkl')

In [61]:
full_list = scgpt + gf

In [67]:
full_list.append('allpairs_spearman_correlation.pkl')

In [70]:
for i in range(5):
    # Step 3: Load features for Fold 0
    train_idx, test_idx = fold_indices[i]
    fold_num = str(i + 1)
    print(f'balanced_train_fold_{fold_num}.csv')
    X_train = load_features_by_index(full_list, train_idx)
    X_train.to_csv(f'balanced_train_fold_{fold_num}.csv')
    print(f'balanced_test_fold_{fold_num}.csv')
    X_test = load_features_by_index(full_list, test_idx)
    X_test.to_csv(f'balanced_test_fold_{fold_num}.csv')

balanced_train_fold_1.csv
balanced_test_fold_1.csv
balanced_train_fold_2.csv
balanced_test_fold_2.csv
balanced_train_fold_3.csv
balanced_test_fold_3.csv
balanced_train_fold_4.csv
balanced_test_fold_4.csv
balanced_train_fold_5.csv
balanced_test_fold_5.csv


In [63]:
# Step 3: Load features for Fold 0
train_idx, test_idx = fold_indices[0]

In [80]:
cols = ['scGPT_bc_embeddings_Cosine_Similarity',
       'scGPT_pancancer_embeddings_Cosine_Similarity',
       'scGPT_CP_embeddings_Cosine_Similarity',
       'scGPT_lung_embeddings_Cosine_Similarity',
       'scGPT_heart_embeddings_Cosine_Similarity',
       'scGPT_brain_embeddings_Cosine_Similarity',
       'scGPT_kidney_embeddings_Cosine_Similarity',
       'scGPT_human_embeddings_Cosine_Similarity',
       'GF-6L30M_HUMANemb_Cosine_Similarity',
       'GF-20L95M_HUMANemb_Cosine_Similarity',
       'GF-12L95M_HUMANemb_Cosine_Similarity',
       'GF-12L95MCANCER_UNIPROT_HUMANemb_Cosine_Similarity',
       'GF-12L30M_HUMANemb_Cosine_Similarity','Correlation','GeneAB', 'Same_Complex']

In [83]:
for i in range(5):
    # Step 3: Load features for Fold 0
    fold_num = str(i + 1)
    fold = str(i)

    X_train = pd.read_csv(f'balanced_train_fold_{fold_num}.csv')
    Y_train = pd.read_csv(f'balanced_train_{fold}_labels.csv')

    train = pd.concat([X_train, Y_train], axis=1)#[cols]
    train.to_csv(f'./balanced_training_test_splits/X_y_train_{fold_num}.csv',index=False)

    X_test = pd.read_csv(f'balanced_test_fold_{fold_num}.csv')
    Y_test = pd.read_csv(f'balanced_test_{fold}_labels.csv')

    test = pd.concat([X_test, Y_test], axis=1)#[cols]
    test.to_csv(f'./balanced_training_test_splits/X_y_test_{fold_num}.csv',index=False)

    
    # train_idx, test_idx = fold_indices[i]
    # fold_num = str(i + 1)
    # print(f'balanced_train_fold_{fold_num}.csv')
    # X_train = load_features_by_index(full_list, train_idx)
    # X_train.to_csv(f'balanced_train_fold_{fold_num}.csv')
    # print(f'balanced_test_fold_{fold_num}.csv')
    # X_test = load_features_by_index(full_list, test_idx)
    # X_test.to_csv(f'balanced_test_fold_{fold_num}.csv')